# Installing Mindcontrol

1. Install meteor

```bash
curl https://install.meteor.com/ | sh
```

2. Clone mindcontrol

```bash
git clone https://github.com/akeshavan/mindcontrol.git
```

3. Start the server

```bash
cd mindcontrol
meteor
```

4. Navigate to http://localhost:3000 in your chrome browser.
You should see the default entries loaded into the mindcontrol database. It should look like mindcontrol.herokuapp.com

<iframe width="420" height="315" src="mindcontrol.herokuapp.com" frameborder="0" allowfullscreen></iframe>
